In [1]:
import uproot as ur
import matplotlib.pyplot as plt
import k3d
import numpy as np
import awkward as ak
import json

In [2]:
eventPath = 'Events.json'

input_file = 'pythia8NCDIS_18x275_minQ2=1000_beamEffects_xAngle=-0.025_hiDiv_1.0000.eicrecon.tree.edm4eic.root'
# Open the root file and get the TTree
events = ur.open("%s:events"%(input_file))
#arrays = events.arrays()

# Get the list of branch names matching the naming pattern
matching_branches = [key for key in events.keys() if key.endswith("RecHits.position.x")]
energy_branches = [key for key in events.keys() if key.endswith("RecHits.energy")]
time_branches = [key for key in events.keys() if key.endswith("RecHits.time")]
print(matching_branches)

['B0ECalRecHits/B0ECalRecHits.position.x', 'B0TrackerRecHits/B0TrackerRecHits.position.x', 'BackwardMPGDEndcapRecHits/BackwardMPGDEndcapRecHits.position.x', 'EcalBarrelImagingRecHits/EcalBarrelImagingRecHits.position.x', 'EcalBarrelScFiRecHits/EcalBarrelScFiRecHits.position.x', 'EcalBarrelSciGlassRecHits/EcalBarrelSciGlassRecHits.position.x', 'EcalEndcapNRecHits/EcalEndcapNRecHits.position.x', 'EcalEndcapPInsertRecHits/EcalEndcapPInsertRecHits.position.x', 'EcalEndcapPRecHits/EcalEndcapPRecHits.position.x', 'EcalLumiSpecRecHits/EcalLumiSpecRecHits.position.x', 'ForwardMPGDEndcapRecHits/ForwardMPGDEndcapRecHits.position.x', 'HcalBarrelRecHits/HcalBarrelRecHits.position.x', 'HcalEndcapNRecHits/HcalEndcapNRecHits.position.x', 'HcalEndcapPInsertRecHits/HcalEndcapPInsertRecHits.position.x', 'HcalEndcapPRecHits/HcalEndcapPRecHits.position.x', 'LFHCALRecHits/LFHCALRecHits.position.x', 'MPGDBarrelRecHits/MPGDBarrelRecHits.position.x', 'MPGDDIRCRecHits/MPGDDIRCRecHits.position.x', 'OuterMPGDBar

In [3]:
t = []
x = []
y = []
z = []
e = []
mint = []

def has_negative_value(lst):
    for sublist in lst:
        for value in sublist:
            if value < 0:
                #mint.append(value)
                return True
    return False


for key in matching_branches:

    x.append(np.array(events[key]))
    y.append(np.array(events[key.replace(".position.x", ".position.y")]))
    z.append(np.array(events[key.replace(".position.x", ".position.z")]))
    
    if has_negative_value(np.array(events[key.replace("RecHits.position.x", "RecHits.time")])):
        print(key.replace(".position.x", ".time"))
        t.append([np.zeros_like(inner) for inner in np.array(events[key])])
    else:
        t.append(np.array(events[key.replace(".position.x", ".time")]))
    
    if key.replace(".position.x", ".energy") in energy_branches:
        e.append(np.array(events[key.replace(".position.x", ".energy")]))
    else:
        e.append([-1*np.ones_like(inner) for inner in np.array(events[key])])   

x = [[inner for sublist in x for inner in sublist[i]] for i in range(len(x[0]))]
y = [[inner for sublist in y for inner in sublist[i]] for i in range(len(y[0]))]
z = [[inner for sublist in z for inner in sublist[i]] for i in range(len(z[0]))]
t = [[inner for sublist in t for inner in sublist[i]] for i in range(len(t[0]))]
e = [[inner for sublist in e for inner in sublist[i]] for i in range(len(e[0]))]

B0TrackerRecHits/B0TrackerRecHits.time
BackwardMPGDEndcapRecHits/BackwardMPGDEndcapRecHits.time
ForwardMPGDEndcapRecHits/ForwardMPGDEndcapRecHits.time
MPGDBarrelRecHits/MPGDBarrelRecHits.time
OuterMPGDBarrelRecHits/OuterMPGDBarrelRecHits.time
SiBarrelTrackerRecHits/SiBarrelTrackerRecHits.time
SiBarrelVertexRecHits/SiBarrelVertexRecHits.time
SiEndcapTrackerRecHits/SiEndcapTrackerRecHits.time


In [4]:
use_log_scale = True  # Set to True for logarithmic scale, False for linear scale
time_window = 10 # All hits below this time value will be given artificial hit times

data = []

numEvts = 20
if len(x) < 20:
    numEvts = len(x)

for i in range(numEvts):
    event = {
        "infoText": "EIC ePIC 18x275 ep Event #" + str(i),
        "colorScale": {}
    }

    # Filter out energies less than or equal to 0
    valid_energies = [energy for energy in e[i] if energy > 0]

    if valid_energies:
        min_energy = min(valid_energies)
        max_energy = max(valid_energies)

        event["colorScale"]["min"] = float(min_energy)
        event["colorScale"]["max"] = float(max_energy)

        if use_log_scale:
            # Logarithmic scaling
            normalized_energy = [np.log(energy / min_energy) / np.log(max_energy / min_energy) if energy > 0 else energy for energy in e[i]]
        else:
            # Linear scaling
            normalized_energy = [(energy - min_energy) / (max_energy - min_energy) if energy > 0 else energy for energy in e[i]]
    else:
        # If all energies are 0 or less, retain the original values
        normalized_energy = e[i]

    objects = []

    for j in range(len(x[i])):
        if t[i][j] > time_window:
            obj = {
                "type": "hit",
                "time": float(t[i][j]),
                "x": float(x[i][j] / 1000),
                "y": float(y[i][j] / 1000),
                "z": float(z[i][j] / 1000),
                "color": float(normalized_energy[j])
            }
        else:
            dist = np.sqrt((x[i][j] / 1000) ** 2 + (y[i][j] / 1000) ** 2 + (z[i][j] / 1000) ** 2)
            c = 0.299792
            time = dist / c

            obj = {
                "type": "hit",
                "time": float(time),
                "x": float(x[i][j] / 1000),
                "y": float(y[i][j] / 1000),
                "z": float(z[i][j] / 1000),
                "color": float(normalized_energy[j])
            }

        objects.append(obj)

    event["objects"] = objects
    data.append(event)

# Write data to JSON file
with open(eventPath, 'w') as json_file:
    json.dump(data, json_file, indent=2)